<a href="https://colab.research.google.com/github/rafaelrubo/python-geociencias/blob/main/python_geociencias06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt

# Carregando imagem RGB​
img = plt.imread("imagem.tif")

FileNotFoundError: [Errno 2] No such file or directory: 'imagem.tif'

In [ ]:
# Separar os canais da imagem
R = img[:, :, 0]
G = img[:, :, 1]
B = img[:, :, 2]

# Filtro Sobel (realce de bordas)
sobel_R = sobel(R)
sobel_G = sobel(G)
sobel_B = sobel(B)

# Filtro Gaussiano (suavização, sigma=1)
gauss_R = gaussian_filter(R, sigma=1)
gauss_G = gaussian_filter(G, sigma=1)
gauss_B = gaussian_filter(B, sigma=1)

# Diferença de Gaussianas (DoG = Gauss(sigma=1) - Gauss(sigma=2))
dog_R = gaussian_filter(R, sigma=1) - gaussian_filter(R, sigma=2)
dog_G = gaussian_filter(G, sigma=1) - gaussian_filter(G, sigma=2)
dog_B = gaussian_filter(B, sigma=1) - gaussian_filter(B, sigma=2)

# Aprox. da Hessiana: traço da matriz Hessiana via Laplaciano (segunda derivada)
hess_R = laplace(R)
hess_G = laplace(G)
hess_B = laplace(B)

# Empilhar todos os atributos por pixel (15 variáveis)
X = np.stack([
    R.ravel(), G.ravel(), B.ravel(),                          # 3
    sobel_R.ravel(), sobel_G.ravel(), sobel_B.ravel(),        # 3
    gauss_R.ravel(), gauss_G.ravel(), gauss_B.ravel(),        # 3
    dog_R.ravel(), dog_G.ravel(), dog_B.ravel(),              # 3
    hess_R.ravel(), hess_G.ravel(), hess_B.ravel()            # 3
], axis=1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Máscara de pixels rotulados
mask = np.zeros((10, 10), dtype=bool)
pixels_anotados = [
    (0, 0), (0, 1), (1, 0), (1, 1),  # vegetação
    (5, 6), (5, 7), (5, 8), (5, 9),  # solo exposto
    (0, 3), (0, 4), (1, 3), (1, 4)  # água
]
for i, j in pixels_anotados:
    mask[i, j] = True

In [ ]:
# Rótulos correspondentes aos pixels anotados
labels = np.array([
    0, 0, 0, 0,        # vegetação
    1, 1, 1, 1,        # solo exposto
    2, 2, 2, 2        # água
])

In [ ]:
# Preparar atributos por pixel
X = X.reshape(-1, 15)                  # vetor (100, 3)
mask_flat = mask.ravel()               # vetor (100,)
X_train = X[mask_flat]                 # atributos apenas dos rotulados
y_train = labels                       # rótulos 1D

In [ ]:
# Preparar atributos por pixel
X = X.reshape(-1, 15)                  # vetor (100, 3)
mask_flat = mask.ravel()               # vetor (100,)
X_train = X[mask_flat]                 # atributos apenas dos rotulados
y_train = labels                       # rótulos 1D

In [ ]:

# Previsão para todos os pixels
y_pred = clf.predict(X)
segmented = y_pred.reshape(10, 10)

In [ ]:
# Plot
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(img)
axes[0].set_title("Imagem RGB")
axes[0].axis('off')

im = axes[1].imshow(segmented, cmap='Accent', vmin=0, vmax=2)
axes[1].set_title("Segmentação (Random Forest)")
axes[1].axis('off')

plt.colorbar(im, ax=axes[1], ticks=[0, 1, 2], label='Classe')
plt.tight_layout()
plt.show()